# ***Methodology***

### Model Architecture

The model architecture adapts large language models (LLMs) for time series forecasting by treating sequential ticket volumes like a language modeling problem : enabling the use of transformer attention to model temporal dependencies.

---

### Sequence Input as Embeddings

Daily ticket volumes and engineered features are converted into **learned time based embeddings** that preserve both numeric magnitude and temporal position. These embeddings are fed into a pre-trained Qwen2.5-0.5B model using `inputs_embeds`, and the transformer backbone is adapted via **LoRA fine-tuning**.

**Why:** Embedding time series as tokens makes it possible to leverage the pretrained knowledge and pattern recognition of LLMs while adapting efficiently with just a few trainable parameters (via LoRA).

---

### TimeAwareEmbed Module

The `TimeAwareEmbed` module combines multiple streams of input:

- **Value Embeddings**: A 2-layer MLP processes the normalized ticket count for each day.
- **Numeric Feature Projections**: Includes lag features, rolling averages, backlog_gap, and resolve_ratio — all passed through a linear layer.
- **Categorical Embeddings**: Calendar features like day-of-week, month, weekend flags, end-of-month, and end-of-quarter are embedded.
- **Positional Encoding**: Sinusoidal encoding preserves the relative order of the sequence.

**Why:** This modular embedding design ensures that each day’s input reflects both historical patterns and operational context, for accurate prediction.

---

### Feature Engineering

Domain-specific features are crafted to reflect customer support dynamics:

- **Lag Features**: Capture short-term memory and recency effects.
- **Rolling Statistics**: Smooth high variance in daily volumes.
- **Backlog Gap**: Measures strain on the support system.
- **Resolve Ratio**: Represents operational throughput efficiency.

**Why:** These features inject domain knowledge directly into the model, improving generalization especially in sparse or volatile regimes.

---

### Quantile Forecasting Head

After the transformer processes the input sequence, a learned `<PRED>` token is appended. Its hidden state is passed through a small MLP to predict **[P10, P50, P90]** quantiles using the **pinball loss function**.

**Why:** Predicting quantiles (instead of just point estimates) gives actionable uncertainty intervals for decisions, making the model robust to outliers and variability.

---

### Training & Setup

- Training windows: **28-day history → next-day forecast**
- Evaluation metrics: MAE, RMSE, Coverage@80%
- Optimization: **LoRA** fine-tuning (low-rank adapters)
- Quantization: Optional **4-bit** (to run on consumer GPUs)
- Backbones: TinyLlama (for CPU/Mac), Qwen2.5/Mistral (for CUDA GPUs)



---

# 1) Data Processing

# i) Helper Functions

> This cell sets up paths, feature definitions, and utility functions, and defines a fit_scalers function to normalize targets and numeric features using a log1p transform for stability.

### Numeric features (`NUMERIC_FEATS`)
- **tickets_received / tickets_resolved** → raw daily counts; the core signal.  
- **lag1, lag7** → ticket volume from 1 day ago or 7 days ago; capture immediate and weekly seasonality.  
- **r7, r14** → 7-day and 14-day rolling averages; smooth out short-term fluctuations and highlight weekly/bi-weekly trends.  
- **backlog_gap** = avg difference between received and resolved over 7 days; indicates whether unresolved tickets are piling up.  
- **resolve_ratio** = resolved ÷ received (7-day sums); shows efficiency of the support team.  

### Categorical features (`CATEG_FEATS`)
- **dow (day of week)** → captures weekday vs. weekend patterns.  
- **dom (day of month)** → useful for month-end surges (e.g., billing cycles).  
- **month** → seasonal effects across months.  
- **is_weekend** → binary flag for low-activity weekends.  
- **eom (end of month)** → spikes in tickets near billing/reporting deadlines.  
- **eoq (end of quarter)** → spikes near quarterly cycles, often tied to finance or compliance.  











In [1]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
from transformers import AutoModelForCausalLM, AutoConfig
from peft import LoraConfig, get_peft_model



OPEN = "requests_opened_external.csv"
CLOSED = "requests_closed_external.csv"
ART_DIR = "artifacts"
os.makedirs(ART_DIR, exist_ok=True)


# Forecasting target and feature names
TARGET = "tickets_received"
NUMERIC_FEATS = [
    "tickets_received", "tickets_resolved",
    "tickets_received_lag1", "tickets_received_lag7", "tickets_received_r7", "tickets_received_r14",
    "tickets_resolved_lag1", "tickets_resolved_lag7", "tickets_resolved_r7", "tickets_resolved_r14",
    "backlog_gap", "resolve_ratio"
]
CATEG_FEATS = ["dow", "dom", "month", "is_weekend", "eom", "eoq"]  # cardinalities: 7,32,12,2,2,2


def detect_device():
    if torch.cuda.is_available(): return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available(): return torch.device("mps")
    return torch.device("cpu")


def save_fig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    print("Saved figure →", path)


def maybe_prepare_kbit(model):
    try:
        from peft import prepare_model_for_kbit_training
        return prepare_model_for_kbit_training(model)
    except Exception:
        return model


def fit_scalers(train_df):
    """
    Compute mean/std for target and numeric features
    so we can normalize consistently.
    """
    # target column - use log1p with NaN handling
    y_vals = train_df[TARGET].values.astype(float)
    y_vals = np.clip(y_vals, 0, None)  # ensure non-negative before log
    logy = np.log1p(y_vals)
    y_mean, y_std = logy.mean(), logy.std() + 1e-8

    # numeric feature columns - use global NUMERIC_FEATS
    num_data = train_df[NUMERIC_FEATS].copy()
    # Handle negative values before log1p
    num_data = num_data.clip(lower=0)
    num_data = num_data.fillna(0)  # handle any remaining NaN

    num_means = np.log1p(num_data).mean()
    num_stds = np.log1p(num_data).std().replace(0, 1)

    return y_mean, y_std, num_means, num_stds



## ii) Data Quality - Audit + Clean



> Loads raw minute-level opened data, coerces numeric volumes and parsed datetimes, drops invalid rows, clips negatives, and aggregates duplicate timestamps; then resamples to daily (tickets_received), reports missing days, generates quick visuals (last-30d series, histogram, daily + 7d MA, weekday boxplot), saves cleaned minute/daily CSVs, prints a DQ summary, and returns both the cleaned minute-level and daily DataFrames.



In [2]:

from typing import Tuple


def dq_report_opened(open_path=OPEN, out_dir=ART_DIR, preview_days=30) -> Tuple[pd.DataFrame, pd.DataFrame]:
    print("\n=== DQ: Loading raw opened data ===")
    raw = pd.read_csv(open_path)
    print(f"Rows (raw): {len(raw)}")
    cols = {c.lower().strip(): c for c in raw.columns}
    need = ["date", "time", "volume"]
    for n in need:
        if n not in cols:
            raise ValueError(f"Missing expected column '{n}'. Found: {list(raw.columns)}")
    DateCol, TimeCol, VolCol = cols["date"], cols["time"], cols["volume"]


    raw["_vol"] = pd.to_numeric(raw[VolCol], errors="coerce")
    bad_vol = raw["_vol"].isna().sum()
    if bad_vol:
        print(f"[WARN] {bad_vol} rows have non-numeric Volume → will drop.")


    dt = pd.to_datetime(
        raw[DateCol].astype(str).str.strip() + " " + raw[TimeCol].astype(str).str.strip(),
        errors="coerce", format="%m/%d/%Y %H:%M"
    )
    bad_dt = dt.isna().sum()
    if bad_dt:
        print(f"[WARN] {bad_dt} rows have invalid Date/Time → will drop.")


    ro = pd.DataFrame({"DateTime": dt, "Volume": raw["_vol"]}).dropna(subset=["DateTime","Volume"]).copy()


    neg = (ro["Volume"] < 0).sum()
    if neg:
        print(f"[WARN] {neg} negative volumes → clipping to 0.")
    ro["Volume"] = ro["Volume"].clip(lower=0)


    dupe_counts = ro.duplicated(subset=["DateTime"]).sum()
    if dupe_counts:
        print(f"[INFO] {dupe_counts} duplicate minute timestamps → aggregating by sum.")
    ro = ro.groupby("DateTime", as_index=False)["Volume"].sum()


    ro = ro.sort_values("DateTime").reset_index(drop=True)
    print(f"Rows (cleaned): {len(ro)} | Range: {ro['DateTime'].min()} → {ro['DateTime'].max()}")


    # Visualizations
    recent_start = ro["DateTime"].max() - pd.Timedelta(days=preview_days)
    ro_recent = ro[ro["DateTime"] >= recent_start]
    plt.figure(figsize=(10,3))
    plt.plot(ro_recent["DateTime"], ro_recent["Volume"])
    plt.title(f"Opened (minute) – last {preview_days} days"); plt.xlabel("Time"); plt.ylabel("Vol/min")
    plt.gcf().autofmt_xdate()
    save_fig(os.path.join(out_dir, f"dq_opened_minute_last{preview_days}d.png"))


    vmax = np.nanpercentile(ro["Volume"], 99.5)
    plt.figure(figsize=(5,3))
    plt.hist(ro["Volume"].clip(upper=vmax), bins=50)
    plt.title("Minute volumes (99.5% clipped)"); plt.xlabel("Vol/min"); plt.ylabel("Count")
    save_fig(os.path.join(out_dir, "dq_opened_minute_hist.png"))


    opened_daily = ro.set_index("DateTime")["Volume"].resample("D").sum().to_frame("tickets_received")
    opened_daily.index.name = "date"


    idx_full = pd.date_range(opened_daily.index.min(), opened_daily.index.max(), freq="D")
    missing_days = idx_full.difference(opened_daily.index)
    print(f"Missing calendar days: {len(missing_days)}")
    if len(missing_days):
        print("First 10 missing:", list(missing_days[:10]))


    plt.figure(figsize=(10,3))
    plt.plot(opened_daily.index, opened_daily["tickets_received"], label="Daily sum")
    plt.plot(opened_daily.index, opened_daily["tickets_received"].rolling(7, min_periods=1).mean(), label="7d mean")
    plt.title("Daily tickets_received"); plt.xlabel("Date"); plt.ylabel("Tickets/day"); plt.legend()
    plt.gcf().autofmt_xdate()
    save_fig(os.path.join(out_dir, "dq_opened_daily_line.png"))


    ddf = opened_daily.copy()
    ddf["dow"] = ddf.index.dayofweek
    data_by_dow = [ddf[ddf["dow"]==i]["tickets_received"].values for i in range(7)]
    plt.figure(figsize=(6,3))
    plt.boxplot(data_by_dow, tick_labels=["Mon","Tue","Wed","Thu","Fri","Sat","Sun"], showfliers=False)
    plt.title("Tickets by weekday"); plt.ylabel("Tickets/day")
    save_fig(os.path.join(out_dir, "dq_opened_daily_box_by_dow.png"))


    minute_clean_path = os.path.join(out_dir, "requests_opened_external_clean.csv")
    daily_clean_path  = os.path.join(out_dir, "opened_daily_clean.csv")
    ro.to_csv(minute_clean_path, index=False)
    opened_daily.to_csv(daily_clean_path)
    print("Saved cleaned minute →", minute_clean_path)
    print("Saved cleaned daily  →", daily_clean_path)


    print("\n=== DQ Summary ===")
    print(f"- Dropped invalid Date/Time rows: {bad_dt}")
    print(f"- Dropped non-numeric Volume rows: {bad_vol}")
    print(f"- Negative volumes clipped: {neg}")
    print(f"- Duplicate minutes aggregated: {dupe_counts}")
    print(f"- Missing calendar days: {len(missing_days)}")
    return ro, opened_daily

## iii) Merge + Feature Engineering (daily)

- **`load_merge_daily`**  
  - Aggregates opened tickets to daily counts.  
  - Clips extreme outliers at the 98th percentile.  
  - Aligns with closed tickets dataset on a full daily index.  
  - Fills gaps (≤7 days) with interpolation + forward/backward fill.  
  - Clips any negative values to 0.  

- **`make_features`**  
  - Adds calendar features: day-of-week, day-of-month, month, weekend flag, end-of-month, end-of-quarter.  
  - Creates lag features (`lag1`, `lag7`) and rolling means (`r7`, `r14`) for opened/closed tickets.  
  - Builds derived features: backlog gap and resolve ratio.  
  - Fills NaNs, runs validation checks, and returns the clean feature matrix.  


In [3]:

def load_merge_daily(open_path=OPEN, closed_path=CLOSED, clip_pct=0.98):  # More aggressive clipping
    ro = pd.read_csv(open_path)
    rc = pd.read_csv(closed_path)

    ro["DateTime"] = pd.to_datetime(ro["Date"] + " " + ro["Time"], format="%m/%d/%Y %H:%M")
    opened_daily = ro.set_index("DateTime")["Volume"].resample("D").sum().to_frame("tickets_received")
    opened_daily.index.name = "date"

    # More aggressive clipping and logging
    before_clip = opened_daily["tickets_received"].max()
    q = opened_daily["tickets_received"].quantile(clip_pct)
    opened_daily["tickets_received"] = opened_daily["tickets_received"].clip(upper=q)
    after_clip = opened_daily["tickets_received"].max()
    print(f"[INFO] Clipped daily max from {before_clip:.0f} to {after_clip:.0f} at {clip_pct:.1%} quantile")

    rc["date"] = pd.to_datetime(rc["date"], format="%Y-%m-%d")
    closed_daily = rc.rename(columns={"volume": "tickets_resolved"}).set_index("date")

    idx = pd.date_range(start=min(opened_daily.index.min(), closed_daily.index.min()),
                        end=max(opened_daily.index.max(), closed_daily.index.max()), freq="D")
    df = opened_daily.reindex(idx).join(closed_daily.reindex(idx), how="outer")
    df = df.interpolate(limit=7, limit_direction="both").ffill().bfill()
    df["tickets_received"] = df["tickets_received"].clip(lower=0)
    df["tickets_resolved"] = df["tickets_resolved"].clip(lower=0)
    return df


def make_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    # Calendar categoricals (zero-based + clipped)
    d["dow"]   = d.index.dayofweek.astype(int)                          # 0..6
    d["dom"]   = np.clip(d.index.day.astype(int)   - 1, 0, 30).astype(int)  # 0..30
    d["month"] = np.clip(d.index.month.astype(int) - 1, 0, 11).astype(int)  # 0..11

    # Binary calendar flags
    d["is_weekend"] = (d["dow"] >= 5).astype(int)
    d["eom"]        = d.index.is_month_end.astype(int)
    d["eoq"]        = d.index.is_quarter_end.astype(int)

    # Lag/rolling features with better NaN handling
    for col in ["tickets_received", "tickets_resolved"]:
        d[f"{col}_lag1"] = d[col].shift(1)
        d[f"{col}_lag7"] = d[col].shift(7)
        d[f"{col}_r7"]   = d[col].rolling(7,  min_periods=1).mean()
        d[f"{col}_r14"]  = d[col].rolling(14, min_periods=1).mean()

    # Backlog & ratio features
    d["backlog_gap"] = (d["tickets_received"] - d["tickets_resolved"]).rolling(7, min_periods=1).mean()
    d["resolve_ratio"] = (d["tickets_resolved"].rolling(7, min_periods=1).sum() + 1) / \
                         (d["tickets_received"].rolling(7, min_periods=1).sum() + 1)

    # Fill any remaining NaN values before dropping
    d = d.fillna(method='ffill').fillna(0)

    # Validation checks
    print(f"[INFO] Features shape before dropna: {d.shape}")
    print(f"[INFO] NaN count by column: {d.isnull().sum().sum()}")

    return d.dropna()

## iv) Dataset & Scalers

- **Target scaling**: `norm_y` applies `log1p` + z-normalization; `denorm_y` reverses it.  
- **Feature scaling**: `transform_numeric` clips negatives, fills NaNs, applies `log1p`, then standardizes with precomputed means/stds.  
- **`TSDataset`**: builds sliding windows of **28 days** (`window`) to predict the **next day** (`horizon=1`), returning numeric features, categorical features, normalized past targets, and the next-day target.  
- **Pipeline**:  
  - Load merged daily data → build engineered features.  
  - Split 80/20 into train/validation sets.  
  - Fit scalers on training data.  
  - Create `train_ds` / `valid_ds` and wrap them in **DataLoaders** (batch=32).  
- **Output**: ~10k days total → ~8k training samples, ~2k validation samples, with logs showing target ranges and normalization stats.  


In [4]:

import math

def norm_y(y, y_mean, y_std):
    y_safe = np.clip(y.astype(np.float32), 0, None)  # ensure non-negative
    return (np.log1p(y_safe) - y_mean) / y_std

def denorm_y(y, y_mean, y_std):
    return np.expm1(y * y_std + y_mean)


# def norm_y(y, y_mean=None, y_std=None):
#     return np.log1p(y.astype(np.float32))

# def denorm_y(y, y_mean=None, y_std=None):
#     return np.expm1(y)

# def fit_scalers(train_df):
#     # Return dummy values since we're not using them
#     return 0.0, 1.0, num_means, num_stds  # Keep numeric feature scaling

# def fit_scalers(train_df: pd.DataFrame):
#     # No longer need y_mean, y_std for target
#     y_mean, y_std = 0.0, 1.0  # dummy values

#     # Still need numeric feature scaling
#     num_data = train_df[NUMERIC_FEATS].copy()
#     num_data = num_data.clip(lower=0).fillna(0)
#     num_means = np.log1p(num_data).mean()
#     num_stds = np.log1p(num_data).std().replace(0, 1)

#     return y_mean, y_std, num_means, num_stds

def transform_numeric(df_part, num_means, num_stds):
    # Clip and fill before log transformation
    X = df_part[NUMERIC_FEATS].copy()
    X = X.clip(lower=0).fillna(0)
    X = np.log1p(X)
    X = (X - num_means) / (num_stds + 1e-8)
    return X.astype(np.float32).values


class TSDataset(Dataset):
    def __init__(self, df_part, y_mean, y_std, num_means, num_stds, window=28, horizon=1):
        self.df = df_part
        self.window = window; self.horizon = horizon
        self.numeric = transform_numeric(df_part, num_means, num_stds)
        self.cats = df_part[CATEG_FEATS].astype(int).values
        self.target = df_part[TARGET].values.astype(np.float32)
        self.y_mean, self.y_std = y_mean, y_std
        self.idx = [i for i in range(window, len(df_part)-horizon)]

        # Validation
        print(f"[INFO] Dataset created: {len(self.idx)} samples, target range: {self.target.min():.1f}-{self.target.max():.1f}")

    def __len__(self): return len(self.idx)
    def __getitem__(self, i):
        t = self.idx[i]; s = t - self.window; e = t
        X_num = torch.tensor(self.numeric[s:e], dtype=torch.float32)
        X_cat = torch.tensor(self.cats[s:e], dtype=torch.long)
        val_series = self.target[s:e]
        val_norm = torch.tensor(norm_y(val_series, self.y_mean, self.y_std), dtype=torch.float32)
        y_next = torch.tensor(norm_y(self.target[t:t+self.horizon], self.y_mean, self.y_std), dtype=torch.float32)
        return {"X_num": X_num, "X_cat": X_cat, "val_norm": val_norm, "y_next": y_next}


# Load and process data
df_daily = load_merge_daily()
feats = make_features(df_daily)
print("Feature frame shape:", feats.shape)

# Split and build loaders
WINDOW, HORIZON = 28, 1
split = int(len(feats)*0.8)
train_df, valid_df = feats.iloc[:split], feats.iloc[split:]

print(f"[INFO] Train target stats: min={train_df[TARGET].min():.1f}, max={train_df[TARGET].max():.1f}, mean={train_df[TARGET].mean():.1f}")
print(f"[INFO] Valid target stats: min={valid_df[TARGET].min():.1f}, max={valid_df[TARGET].max():.1f}, mean={valid_df[TARGET].mean():.1f}")

y_mean, y_std, num_means, num_stds = fit_scalers(train_df)
print(f"[INFO] Normalization params: y_mean={y_mean:.3f}, y_std={y_std:.3f}")

train_ds = TSDataset(train_df, y_mean, y_std, num_means, num_stds, window=WINDOW, horizon=HORIZON)
valid_ds = TSDataset(valid_df, y_mean, y_std, num_means, num_stds, window=WINDOW, horizon=HORIZON)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False)

[INFO] Clipped daily max from 134993461 to 24344049 at 98.0% quantile
[INFO] Features shape before dropna: (10045, 18)
[INFO] NaN count by column: 0
Feature frame shape: (10045, 18)
[INFO] Train target stats: min=0.0, max=24344048.8, mean=6837270.6
[INFO] Valid target stats: min=0.0, max=24344048.8, mean=2241675.8
[INFO] Normalization params: y_mean=10.978, y_std=7.395
[INFO] Dataset created: 8007 samples, target range: 0.0-24344048.0
[INFO] Dataset created: 1980 samples, target range: 0.0-24344048.0


/tmp/ipython-input-177649642.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  d = d.fillna(method='ffill').fillna(0)


## v) Category Sizes

- Computes the number of unique values for each categorical feature, to define embedding dimensions later.  
- Results:  
  - `dow`: 7 (days of week)  
  - `dom`: 31 (days of month)  
  - `month`: 12 (months)  
  - `is_weekend`: 2  
  - `eom`: 2 (end-of-month flag)  
  - `eoq`: 2 (end-of-quarter flag)  
- **Output**: `[7, 31, 12, 2, 2, 2]` → used to size categorical embedding layers.  


In [5]:
cat_sizes = [
    feats["dow"].max() + 1,     # 7
    feats["dom"].max() + 1,     # 31
    feats["month"].max() + 1,   # 12
    feats["is_weekend"].max() + 1,  # 2
    feats["eom"].max() + 1,         # 2
    feats["eoq"].max() + 1,         # 2
]
print("cat_sizes:", cat_sizes)

cat_sizes: [7, 31, 12, 2, 2, 2]


---

## Data Quality Findings & Decisions

**Issue 1: Extreme Outliers (135M → 24M tickets/day)**
- Decision: Clip at 98th percentile rather than 99.5%
- Rationale: Values >100M appear to be data entry errors rather than genuine volume spikes
- Impact: Removes 2% of extreme values while preserving realistic high-volume periods

**Issue 2: Weekend vs Weekday Patterns**
- Finding: Weekends show near-zero activity.
- Decision: Preserve this pattern rather than smooth it
- Rationale: Reflects genuine operational reality for staffing decisions

**Issue 3: Feature Normalization Failures**

Finding: Multiple RuntimeWarning: invalid value encountered in log1p during feature scaling
Root cause: Negative values in lag/rolling features creating NaN after log transformation
Decision: Added explicit .clip(lower=0) before log1p transformation
Rationale: Negative "resolved tickets" don't make business sense; likely interpolation artifacts

**Issue 4: Z-Score Normalization Amplification Problem**

Finding: Small prediction errors in normalized space (±0.1) became massive real-world errors (millions of tickets)
Analysis: expm1(z * 7.4 + 11.0) exponential denormalization amplifies any prediction uncertainty
Decision: Acknowledged limitation rather than masking it with artificial constraints
Alternative considered: Direct log-space prediction (tested but showed poor learning)

**Issue 5: Target Distribution Extreme Skewness**

Finding: Target values spanning 0 to 135M+ tickets (8+ orders of magnitude)
Investigation: Single day spike to 135M represents ~4x normal maximum volume
Decision: Aggressive 98% clipping rather than 99.5%
Rationale: Balance between preserving genuine high-volume events vs removing apparent data errors
Impact: Reduced max from 135M to 24M while retaining realistic operational range

**Issue 6: Time Series Continuity Problems**

Finding: Minute-level data required aggregation to daily, but no missing timestamps found
Decision: Simple daily sum aggregation rather than complex weighted averaging
Validation: Confirmed 0 missing calendar days in final dataset
Business logic: Daily totals more relevant for staffing than intraday patterns

**Issue 7: Cross-Dataset Integration Challenges**

Finding: Different date formats between opened (M/d/Y H:M) and closed (Y-m-d) datasets
Decision: Standardize both to daily pandas DatetimeIndex for alignment
Gap handling: Interpolate missing closed ticket data with 7-day limit
Rationale: Closed tickets less critical than opened for forecasting, but useful for operational features

**Issue 8: Feature Engineering Validation**

Finding: Initial feature definitions in fit_scalers() didn't match actual engineered features
Problem: Hardcoded feature list caused crashes during training
Decision: Refactored to use global NUMERIC_FEATS consistently
Prevention: Added validation prints showing feature shapes and NaN counts

**Issue 9: Model Architecture vs Data Scale Mismatch**

Finding: 500M parameter model for 8K training samples (62,500:1 parameter-to-sample ratio)
Analysis: Massive overparameterization typically leads to overfitting
Mitigation: LoRA fine-tuning (only 0.88% parameters trainable) + aggressive regularization
Monitoring: Close validation performance to training indicates acceptable generalization

**Issue 10: Evaluation Pipeline Architectural Decisions**

Challenge: How to evaluate a model trained in normalized space against business requirements
Decision: Dual evaluation approach - normalized space (for model assessment) + denormalized space (for baseline comparison)
Trade-off: Accurate model performance measurement vs interpretable business metrics
Documentation: Explicit acknowledgment that current pipeline creates evaluation challenges for production deployment

---



# 2) Model Architechture

## i) Model: Time-Aware Embeddings → LLM Backbone → Quantile Forecasts

- **`TimeAwareEmbed`**: builds per-day embeddings by combining (a) `value_mlp(val_norm)` for past target, (b) linear projection of **numeric features**, (c) summed **categorical embeddings** (sizes from `cat_sizes`), plus **sinusoidal positional encodings**; then applies `LayerNorm`, dropout, and NaN/Inf guards.
- **`QuantileHead`**: MLP that outputs ordered **p10/p50/p90** via cumulative softplus widths (`a`, `a+softplus(b)`, `…+softplus(c)`) to guarantee monotonic quantiles; clamps to a safe range.
- **`TSLLM`**: adapts a causal LLM by feeding **inputs_embeds** (time-aware sequence + learned `<PRED>` token), scales with `input_gain`, requests **hidden states**, and reads the final token’s hidden vector to the quantile head.
- **Losses**: `pinball_loss` for quantile regression; `safe_pinball_loss` adds sanitization, clamping, and enforced monotonicity for stable training.


In [6]:


class TimeAwareEmbed(nn.Module):
    def __init__(self, d_model: int, num_numeric: int, cat_sizes=(7,32,12,2,2,2), p_drop=0.1):
        super().__init__()
        self.value_mlp = nn.Sequential(
            nn.Linear(1, d_model),
            nn.ReLU(),
            nn.Linear(d_model, d_model)
        )
        self.num_proj  = nn.Linear(num_numeric, d_model)
        self.cat_embs  = nn.ModuleList([nn.Embedding(s, d_model) for s in cat_sizes])
        self.ln = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(p_drop)
        self.d_model = d_model

    def forward(self, val_norm, num_feats, cat_feats):
        B, W, _ = num_feats.shape

        # categorical range guard
        for i, emb in enumerate(self.cat_embs):
            mx = int(cat_feats[..., i].max().item()); mn = int(cat_feats[..., i].min().item())
            if mx >= emb.num_embeddings or mn < 0:
                raise ValueError(f"Cat feat {i} out of range [{mn},{mx}] vs size {emb.num_embeddings}")

        v_emb = self.value_mlp(val_norm.view(B, W, 1))
        n_emb = self.num_proj(num_feats)
        c_sum = 0
        for i, emb in enumerate(self.cat_embs):
            c_sum = c_sum + emb(cat_feats[..., i])

        # sinusoidal positional encoding
        device = num_feats.device
        pos = torch.arange(W, device=device).unsqueeze(1).float()
        i = torch.arange(self.d_model, device=device).float().unsqueeze(0)
        angle = pos * (1/torch.pow(10000, (2*(i//2))/self.d_model))
        pe = torch.zeros(W, self.d_model, device=device)
        pe[:, 0::2] = torch.sin(angle[:, 0::2]); pe[:, 1::2] = torch.cos(angle[:, 1::2])

        x = v_emb + n_emb + c_sum + pe.unsqueeze(0).expand(B, -1, -1)

        # numeric stability: replace NaN/Inf, scale, norm, and drop
        x = torch.nan_to_num(x, nan=0.0, posinf=1e6, neginf=-1e6)
        x = x * (1.0 / math.sqrt(self.d_model))
        x = self.ln(x)
        x = self.drop(x)
        return x



import torch.nn.functional as F

class QuantileHead(nn.Module):
    """
    Monotonic head with cumulative softplus widths:
      p10 = a
      p50 = a + softplus(b) + eps
      p90 = p50 + softplus(c) + eps
    This guarantees ordered quantiles and non-zero width.
    """
    def __init__(self, hidden_size: int, horizon=1, quantiles=(0.1,0.5,0.9), eps=1e-3, p_drop=0.1):
        super().__init__()
        hs = hidden_size
        self.mlp = nn.Sequential(
            nn.Linear(hs, 2*hs), nn.GELU(), nn.Dropout(p_drop),
            nn.Linear(2*hs, hs), nn.GELU(), nn.Dropout(p_drop),
        )
        self.out = nn.Linear(hs, horizon*3)   # (a, b, c) per step
        self.horizon = horizon
        self.eps = eps

        # init: small random, slight positive widths so we don't start with zero interval
        nn.init.zeros_(self.out.bias)
        nn.init.normal_(self.out.weight, mean=0.0, std=0.01)

    def forward(self, hidden):
        h = self.mlp(hidden)
        raw = self.out(h).view(-1, self.horizon, 3)  # [..., (a,b,c)]

        a = raw[..., 0]
        b = F.softplus(raw[..., 1]) + self.eps
        c = F.softplus(raw[..., 2]) + self.eps

        p10 = a
        p50 = a + b
        p90 = p50 + c

        pred = torch.stack([p10, p50, p90], dim=-1)  # [B, H, 3]
        # absolute guard in z-space
        pred = torch.nan_to_num(pred, nan=0.0, posinf=1e6, neginf=-1e6)
        pred = torch.clamp(pred, -10.0, 10.0)
        return pred




## ii) TSLLM + Quantile Losses

- **`TSLLM`**: Feeds time-aware embeddings plus a learned `<PRED>` token into the LLM backbone via `inputs_embeds`, scales with `input_gain`, requests hidden states, and takes the final token’s hidden vector to a `QuantileHead` that outputs ordered p10/p50/p90 forecasts.
- **Losses**: `pinball_loss` does standard quantile regression; `safe_pinball_loss` adds NaN/Inf sanitization, clamps values, and enforces monotonic quantiles for stable training.


In [7]:

class TSLLM(nn.Module):
    def __init__(self, base_causal_lm, hidden_size: int,
                 num_numeric: int, cat_sizes=(7,32,12,2,2,2),
                 horizon=1, quantiles=(0.1,0.5,0.9)):
        super().__init__()
        # keep a handle to the full module (could be PEFT-wrapped)
        self.full = base_causal_lm
        # try to grab the underlying backbone if exposed; otherwise use the module itself
        self.backbone = getattr(base_causal_lm, "model", None)
        if self.backbone is None:
            self.backbone = getattr(base_causal_lm, "base_model", base_causal_lm)

        self.embedder = TimeAwareEmbed(hidden_size, num_numeric, cat_sizes)
        self.pred_tok = nn.Parameter(torch.randn(1,1,hidden_size)*0.02)
        self.head = QuantileHead(hidden_size, horizon=horizon, quantiles=quantiles)
        self.input_gain = nn.Parameter(torch.tensor(5.0))

    def forward(self, val_norm, X_num, X_cat):
        # build sequence of learned time-aware embeddings
        x = self.embedder(val_norm, X_num, X_cat)                 # [B, W, d]
        B = x.size(0)
        seq = torch.cat([x, self.pred_tok.repeat(B,1,1)], dim=1) # [B, W+1, d]
        seq = self.input_gain * seq
        # causal mask is handled internally by decoder-only models; we just give a basic attention mask
        attn_mask = torch.ones(seq.size()[:2], dtype=torch.long, device=seq.device)


        out = self.backbone(
            inputs_embeds=seq,
            attention_mask=attn_mask,
            use_cache=False,
            output_hidden_states=True,
            return_dict=True,
        )

        # Robustly get the final hidden representation of the <PRED> token
        if hasattr(out, "last_hidden_state") and out.last_hidden_state is not None:
            hidden_step = out.last_hidden_state[:, -1, :]   # [B, d]
        elif hasattr(out, "hidden_states") and out.hidden_states is not None:
            hidden_step = out.hidden_states[-1][:, -1, :]   # [B, d]
        else:
            # If the model still doesn't return hidden states, raise a helpful error.
            raise RuntimeError(
                "Backbone did not return hidden states. "
                "Ensure `output_hidden_states=True` and that you're calling the base model."
            )

        return self.head(hidden_step)


def pinball_loss(pred, y, quantiles=(0.1,0.5,0.9)):
    diff = y.unsqueeze(-1) - pred
    parts = [torch.maximum(q*diff[...,i], (q-1)*diff[...,i]) for i,q in enumerate(quantiles)]
    return torch.mean(torch.stack(parts, dim=-1))

def safe_pinball_loss(pred, y, quantiles=(0.1, 0.5, 0.9)):
    """
    Robust pinball loss:
      - sanitizes NaN/Inf
      - clamps normalized space to [-10, 10] (prevents exp overflow later)
      - enforces monotonic quantiles (P10<=P50<=P90) to keep gradients sane
    pred: [B, H, Q], y: [B, H]
    """
    # sanitize
    pred = torch.nan_to_num(pred, nan=0.0, posinf=1e6, neginf=-1e6)
    y    = torch.nan_to_num(y,    nan=0.0, posinf=1e6, neginf=-1e6)

    # clamp in normalized space
    pred = torch.clamp(pred, -10.0, 10.0)
    y    = torch.clamp(y,   -10.0, 10.0)

    # enforce monotonic quantiles
    pred, _ = torch.sort(pred, dim=-1)

    u = y.unsqueeze(-1) - pred  # [B, H, Q]
    losses = []
    for i, q in enumerate(sorted(quantiles)):
        left  = q * u[..., i]
        right = (q - 1.0) * u[..., i]
        term = torch.maximum(left, right)
        term = torch.nan_to_num(term, nan=0.0, posinf=1e6, neginf=-1e6)
        losses.append(term.mean())
    return sum(losses) / len(losses)



## iii) Build & Train Setup

- **Backbone & PEFT**: Loads a small causal LM (`Qwen2.5-0.5B-Instruct` by default; optional Mistral + 4-bit), applies **LoRA** to attention/MLP proj layers, and reports trainable params.
- **TSLLM wrapper**: Instantiates `TSLLM` with `hidden_size` from config, numeric feature count, and computed `cat_sizes`; forecasts `p10/p50/p90` for horizon=1.
- **Optimizer**: AdamW on trainable (LoRA) params with `lr=2e-4`, `weight_decay=1e-4`.
- **`evaluate(...)`**: Runs model in eval mode, clamps/sanitizes quantiles, **de-normalizes** outputs, filters non-finite values, and computes **MAE**, **RMSE**, and **P10–P90 coverage**; also returns arrays for plotting.


In [8]:

use_mistral = False
model_id = "Qwen/Qwen2.5-0.5B-Instruct" if not use_mistral else "mistralai/Mistral-7B-Instruct-v0.2"
use_4bit = False if not use_mistral else True

device = detect_device()
print("Device:", device)
cfg = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
load_kwargs = dict(trust_remote_code=True)
if use_4bit and device.type == "cuda":
    load_kwargs.update(dict(device_map="auto", load_in_4bit=True))
base_lm = AutoModelForCausalLM.from_pretrained(model_id, **load_kwargs)
if use_4bit and device.type == "cuda":
    base_lm = maybe_prepare_kbit(base_lm)

lora = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)
base_lm = get_peft_model(base_lm, lora)
base_lm.print_trainable_parameters()

ts_model = TSLLM(
    base_causal_lm=base_lm,
    hidden_size=cfg.hidden_size,
    num_numeric=len(NUMERIC_FEATS),
    cat_sizes=cat_sizes,           # <--- use sanitized, computed sizes
    horizon=1, quantiles=(0.1,0.5,0.9)
).to(device)

optim = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, ts_model.parameters()),
    lr=2e-4, weight_decay=1e-4
)

def evaluate(model, loader, y_mean, y_std, device):
    model.eval()
    pq_list, yt_list = [], []

    with torch.no_grad():
        for b in loader:
            out = model(
                b["val_norm"].to(device),
                b["X_num"].to(device),
                b["X_cat"].to(device),
            )
            # Stabilize normalized quantiles before de-normalization
            out = torch.clamp(out, -10.0, 10.0)                     # <- important
            out = torch.nan_to_num(out, nan=0.0, posinf=1e6, neginf=-1e6)
            pq_list.append(out.cpu().numpy())
            yt_list.append(b["y_next"].cpu().numpy())

    pq = np.concatenate(pq_list, 0)   # [N, H, 3]
    yt = np.concatenate(yt_list, 0)   # [N, H]

    # squeeze horizon=1
    p10n = pq[..., 0].reshape(-1)
    p50n = pq[..., 1].reshape(-1)
    p90n = pq[..., 2].reshape(-1)
    yn   = yt.reshape(-1)

    # de-normalize
    p10 = denorm_y(p10n, y_mean, y_std)
    p50 = denorm_y(p50n, y_mean, y_std)
    p90 = denorm_y(p90n, y_mean, y_std)
    y   = denorm_y(yn,   y_mean, y_std)

    # mask out non-finite values so sklearn never sees NaN/Inf
    mask = np.isfinite(p10) & np.isfinite(p50) & np.isfinite(p90) & np.isfinite(y)
    if mask.sum() < mask.size:
        bad = mask.size - mask.sum()
        print(f"[evaluate] filtered {bad}/{mask.size} non-finite points (after denorm).")
        # optional quick counts:
        # print("finite:", dict(y=np.isfinite(y).sum(), p10=np.isfinite(p10).sum(),
        #                      p50=np.isfinite(p50).sum(), p90=np.isfinite(p90).sum()))

    y_f, p10_f, p50_f, p90_f = y[mask], p10[mask], p50[mask], p90[mask]
    mae = mean_absolute_error(y_f, p50_f)
    mse = mean_squared_error(y_f, p50_f)   # works on all versions
    rmse = np.sqrt(mse)

    cov  = np.mean((y_f >= p10_f) & (y_f <= p90_f)) if len(y_f) else np.nan
    return mae, rmse, cov, y_f, p50_f, p10_f, p90_f

print("Model loading complete")

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826
Model loading complete


In [9]:
# Sanity check: a single forward + finiteness
b = next(iter(train_loader))
with torch.no_grad():
    out = ts_model(b["val_norm"].to(device), b["X_num"].to(device), b["X_cat"].to(device))
print("out shape:", tuple(out.shape))
print("any NaN in raw out?", torch.isnan(out).any().item(),
      "any Inf?", torch.isinf(out).any().item())


out shape: (32, 1, 3)
any NaN in raw out? False any Inf? False


## iv) Normalized Evaluation (helper function)

Evaluates the model **in training (z-) space**: gets quantile preds (p10/p50/p90) and targets already normalized, sanitizes/clamps/sorts them, then reports **MAE_z**, **RMSE_z**, and **80% coverage** without de-normalizing. Returns arrays for optional plots/diagnostics.


In [10]:
# =======================
# Normalized evaluation
# =======================
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import torch

def evaluate_normalized(model, loader, device):
    """
    Evaluate in the SAME normalized space used during training:
      - predictions are the quantiles in z-space (normalized log1p units)
      - target is y_next already normalized by the dataset class
    Reports: MAE_z, RMSE_z, and coverage (80% PI) all in normalized units.
    """
    model.eval()
    preds, ys = [], []

    with torch.no_grad():
        for b in loader:
            out = model(
                b["val_norm"].to(device),  # [B, W]
                b["X_num"].to(device),     # [B, W, F_num]
                b["X_cat"].to(device),     # [B, W, F_cat]
            )
            # keep things safe in z-space
            out = torch.nan_to_num(out, nan=0.0, posinf=1e6, neginf=-1e6)
            out = torch.clamp(out, -10.0, 10.0)
            # enforce monotonic quantiles (P10<=P50<=P90)
            out, _ = torch.sort(out, dim=-1)

            preds.append(out.cpu().numpy())              # [B, H, 3]
            ys.append(b["y_next"].cpu().numpy())         # [B, H]

    preds = np.concatenate(preds, axis=0)               # [N, H, 3]
    ys    = np.concatenate(ys, axis=0)                  # [N, H]

    # collapse horizon=1
    p10n = preds[..., 0].reshape(-1)
    p50n = preds[..., 1].reshape(-1)
    p90n = preds[..., 2].reshape(-1)
    yn   = ys.reshape(-1)

    # mask any stragglers
    m = np.isfinite(p10n) & np.isfinite(p50n) & np.isfinite(p90n) & np.isfinite(yn)
    p10n, p50n, p90n, yn = p10n[m], p50n[m], p90n[m], yn[m]

    mae_z  = mean_absolute_error(yn, p50n)
    rmse_z = mean_squared_error(yn, p50n) ** 0.5
    cov80  = float(np.mean((yn >= p10n) & (yn <= p90n))) if len(yn) else np.nan

    return mae_z, rmse_z, cov80, yn, p50n, p10n, p90n


## Training Loop

- Trains for **5 epochs** with AdamW: forward → clamp preds in z-space → `safe_pinball_loss` → backprop with grad clipping (1.0) → step.
- After each epoch, runs **normalized eval** on train/valid (reports z-MAE, z-RMSE, 80% coverage) and prints a **debug snapshot** of batch quantiles (P10/P50/P90 percentiles) to monitor calibration.


In [12]:
EPOCHS = 5
for e in range(1, EPOCHS+1):
    ts_model.train()
    total = 0.0
    for b in train_loader:
        optim.zero_grad()
        out = ts_model(b["val_norm"].to(device),
                       b["X_num"].to(device),
                       b["X_cat"].to(device))
        out = torch.clamp(out, -10.0, 10.0)
        loss = safe_pinball_loss(out, b["y_next"].to(device))
        if not torch.isfinite(loss):
            continue
        loss.backward()
        nn.utils.clip_grad_norm_(ts_model.parameters(), 1.0)
        optim.step()
        total += float(loss.item())

    tr = evaluate_normalized(ts_model, train_loader, device)
    va = evaluate_normalized(ts_model, valid_loader, device)
    print(f"Epoch {e}: loss={total/max(len(train_loader),1):.4f} | "
          f"train z-MAE={tr[0]:.4f} z-RMSE={tr[1]:.4f} COV80={tr[2]:.2f} || "
          f"valid z-MAE={va[0]:.4f} z-RMSE={va[1]:.4f} COV80={va[2]:.2f}")
    with torch.no_grad():
      dbg = out.detach().cpu().numpy().reshape(-1,3)  # from a batch
      print("pred z-quantiles (batch sample) P10/P50/P90:",
            np.percentile(dbg[:,0], [10,50,90]),
            np.percentile(dbg[:,1], [10,50,90]),
            np.percentile(dbg[:,2], [10,50,90]))


Epoch 1: loss=0.0502 | train z-MAE=0.0922 z-RMSE=0.3291 COV80=0.92 || valid z-MAE=0.0891 z-RMSE=0.3182 COV80=0.94
pred z-quantiles (batch sample) P10/P50/P90: [-1.55677402  0.43040392  0.51835746] [-1.4777895   0.55597201  0.65233166] [-1.39874369  0.65306416  0.7579147 ]
Epoch 2: loss=0.0415 | train z-MAE=0.0887 z-RMSE=0.3155 COV80=0.84 || valid z-MAE=0.0895 z-RMSE=0.2922 COV80=0.84
pred z-quantiles (batch sample) P10/P50/P90: [-1.58486725  0.53067109  0.64163725] [-1.53040472  0.60428077  0.72626367] [-1.43626112  0.67472291  0.77879852]
Epoch 3: loss=0.0406 | train z-MAE=0.0952 z-RMSE=0.3277 COV80=0.80 || valid z-MAE=0.0963 z-RMSE=0.3218 COV80=0.84
pred z-quantiles (batch sample) P10/P50/P90: [-1.61726528  0.50416547  0.63571354] [-1.5847495   0.58893105  0.70747436] [-1.50379894  0.64438543  0.75570304]
Epoch 4: loss=0.0396 | train z-MAE=0.0993 z-RMSE=0.3225 COV80=0.53 || valid z-MAE=0.1001 z-RMSE=0.3085 COV80=0.59
pred z-quantiles (batch sample) P10/P50/P90: [-1.60767783  0.477059

In [13]:
mae, rmse, cov, y, p50, p10, p90 = evaluate(ts_model, valid_loader, y_mean, y_std, device)
print(mae, rmse, cov)


739264.5954921641 1832695.283812338 0.8156565656565656


In [14]:
# 1) Seasonal naïve (t-7) baseline + 80% PI from train residuals
import numpy as np, pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

df_all = pd.concat([train_df, valid_df])
y_all = df_all[TARGET].clip(lower=0)
naive_pred = y_all.shift(7)

# train residuals in log space
train_mask = ~naive_pred.iloc[:len(train_df)].isna()
r_train = np.log1p(y_all.iloc[:len(train_df)][train_mask]) - np.log1p(naive_pred.iloc[:len(train_df)][train_mask])
q10, q90 = np.percentile(r_train, [10, 90])

# valid metrics
valid_idx = valid_df.index
yp = naive_pred.loc[valid_idx].values
ya = y_all.loc[valid_idx].values
mask = np.isfinite(yp) & np.isfinite(ya)
yp, ya = yp[mask], ya[mask]

# build PI (apply residual quantiles to log-preds)
logp = np.log1p(yp)
p10 = np.expm1(logp + q10)
p50 = yp
p90 = np.expm1(logp + q90)

mae = mean_absolute_error(ya, p50)
rmse = mean_squared_error(ya, p50)**0.5
cov80 = np.mean((ya >= p10) & (ya <= p90))

print(f"[Naive t-7] MAE={mae:,.0f}  RMSE={rmse:,.0f}  COV80={cov80:.2f}")


[Naive t-7] MAE=1,160,776  RMSE=2,562,661  COV80=0.75


In [16]:
# Fixed linear baseline: scaled features, no target leakage, Ridge regularization
import numpy as np, pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1) Define numeric features without the target to avoid leakage
NUMERIC_FEATS_BASE = [f for f in NUMERIC_FEATS if f != TARGET]

# 2) Fit scaler (log1p mean/std) on TRAIN ONLY
Xn_tr_raw = train_df[NUMERIC_FEATS_BASE].clip(lower=0).fillna(0).astype(float)
mu = np.log1p(Xn_tr_raw).mean()
sd = np.log1p(Xn_tr_raw).std().replace(0, 1)

def build_X_scaled(df, mu, sd, train_cols=None):
    Xn = df[NUMERIC_FEATS_BASE].clip(lower=0).fillna(0).astype(float)
    Xn = (np.log1p(Xn) - mu) / (sd + 1e-8)
    Xc = pd.get_dummies(df[CATEG_FEATS].astype(int), columns=CATEG_FEATS, drop_first=False)
    X  = pd.concat([Xn, Xc], axis=1).fillna(0)
    if train_cols is not None:
        X = X.reindex(columns=train_cols, fill_value=0)
    return X

X_tr = build_X_scaled(train_df, mu, sd)
train_cols = X_tr.columns
X_va = build_X_scaled(valid_df, mu, sd, train_cols)

# 3) Train Ridge on log1p(target)
y_tr_log = np.log1p(train_df[TARGET].clip(lower=0).values)
ridge = Ridge(alpha=1.0, random_state=0).fit(X_tr, y_tr_log)

# 4) Predictions + 80% PI from train residuals (in log space)
pred_va_log = ridge.predict(X_va)
p50 = np.expm1(pred_va_log)

r_train = y_tr_log - ridge.predict(X_tr)
q10, q90 = np.percentile(r_train, [10, 90])
p10 = np.expm1(pred_va_log + q10)
p90 = np.expm1(pred_va_log + q90)

y_va = valid_df[TARGET].clip(lower=0).values
mae = mean_absolute_error(y_va, p50)
rmse = mean_squared_error(y_va, p50) ** 0.5
cov80 = np.mean((y_va >= p10) & (y_va <= p90))
print(f"[Ridge (fixed)] MAE={mae:,.0f}  RMSE={rmse:,.0f}  COV80={cov80:.2f}  (features={X_tr.shape[1]})")

# Optional: compute lift vs t-7 and vs TSLLM
ts_mae, ts_rmse, ts_cov, *_ = evaluate(ts_model, valid_loader, y_mean, y_std, device)
print(f"Lift vs t-7  → TSLLM MAE: {(1 - 1_160_776/ts_mae):.1%} (use your latest t-7 numbers if different)")
print(f"Lift vs Ridge→ TSLLM MAE: {(mae - ts_mae)/mae:.1%}, RMSE: {(rmse - ts_rmse)/rmse:.1%}")


[Ridge (fixed)] MAE=1,312,100  RMSE=2,635,984  COV80=0.77  (features=67)
Lift vs t-7  → TSLLM MAE: -57.0% (use your latest t-7 numbers if different)
Lift vs Ridge→ TSLLM MAE: 43.7%, RMSE: 30.5%


---

## 3) Results & Baselines


The LLM forecaster (TSLLM) materially outperforms reasonable baselines and is well-calibrated.
- Accuracy: **MAE = 0.739M**, **RMSE = 1.832M**
- Calibration: **COV80 = 0.816** (target ≈ 0.80)

| Model            | MAE        | RMSE       | COV80 | Lift vs t−7 | Lift vs Ridge |
|------------------|------------|------------|:-----:|------------:|--------------:|
| **TSLLM (ours)** | **0.739M** | **1.832M** | 0.816 | —           | —             |
| Seasonal t−7     | 1.161M     | 2.563M     | 0.75  | **−36.3% MAE**, **−28.5% RMSE** | — |
| Ridge (fixed)    | 1.312M     | 2.636M     | 0.77  | —           | **−43.7% MAE**, **−30.5% RMSE** |

*Lift = (baseline − model) / baseline.*

**Interpretation**
- TSLLM generalizes well (train ≈ valid in z-space) and provides **narrower errors** than both baselines.
- **Coverage ~0.82** indicates slightly conservative intervals but close to the 80% target—acceptable for staffing use.



---

## Observation and Next steps:



1.   The model demonstrates effective learning in its training space (normalized log-transformed ticket counts) with well-calibrated uncertainty estimates, achieving 36-44% accuracy improvements over seasonal naive and Ridge regression baselines. However, the current normalization strategy creates challenges for business interpretation, as the exponential denormalization amplifies small prediction errors significantly.
2.   While the performance gains validate the technical approach, production deployment requires weighing these accuracy improvements against operational complexity.
        *   The LLM-based system requires GPU infrastructure, complex retraining procedures, and significant computational overhead compared to simpler baselines that achieve reasonable performance with minimal operational burden.
        *   In many production environments, the interpretability, operational simplicity, and instant retrainability of traditional forecasting methods may outweigh the accuracy benefits of the transformer approach.

3.   For production deployment, I recommend either direct log-space prediction to address the interpretability issues, or a cost-benefit analysis comparing the operational overhead of the LLM system against the business value of the accuracy improvements. The architecture successfully demonstrates that large language models can be adapted for time series forecasting, but practical deployment depends on whether the specific use case justifies the complexity.

---